![alt text](img/MIoT_ML.png "MIoT_ML")
# Unidad 04  Entrenamiento de Modelos de Aprendizaje Automático

El objetivo principal de esta práctica es el desarrollo, optimización y evaluación de un modelo de Aprendizaje Automático.

El Notebook contiene varios ejercicios sencillos. Debéis desarrollarlos durante la clase y subirlos al aula virtual.

## Referencias útiles para la práctica
1. API Pandas: [https://pandas.pydata.org/docs/reference/index.html](https://pandas.pydata.org/docs/reference/index.html)
2. API Scikit-learn: [https://scikit-learn.org/stable/api/index.html](https://scikit-learn.org/stable/api/index.html)
3. Dataset para el ejercicio: [https://www.kaggle.com/datasets/camnugent/california-housing-prices](https://www.kaggle.com/datasets/camnugent/california-housing-prices)
4. Géron, Aurélien. Hands-on machine learning with Scikit-Learn, Keras, and TensorFlow. " O'Reilly Media, Inc.", 2022.
5. Bergstra, J., & Bengio, Y. (2012). Random search for hyperparameter optimization. Journal of machine learning research, 13 (2). *Para profundizar en la optimización de hiperparámetros*
   
## 1. Flujo de trabajo básico en problemas de Aprendizaje Automático (*ML workflow*)
A la hora de enfrentarnos a un nuevo problema de Aprendizaje Automático (ML), existen una serie de pasos típicos y comunes que debemos afrontar:
1. Entender el problema y su contexto.
2. Obtener los datos (histórico).
3. Explorar, analizar y entender los datos.
4. Preparar los datos para los modelos.
5. Seleccionar, optimizar y entrenar los modelos ML.
6. Evaluar y presentar el modelo seleccionado.
7. Desplegar, monitorizar y mantener la solución.

En esta unidad nos centraremos en los pasos 5 y 6 del flujo de trabajo.

## 2. Carga y partición de datos
### 2.1. Carga de los datos
Cargaremos los datos generados durante la Unidad02, almacenados al finalizar la misma. Suponemos que se han mantenido los nombres generados durante dicha unidad, y que los datos están almacenados en el directorio raíz.

- Características de entrada preprocesadas (*Inputs*) de las observaciones del conjunto de entrenamiento (*preprocessing_trainset_inputs.csv*).
- Etiquetas de salida (*Outputs*) de las observaciones del conjunto de entrenamiento (*trainset_ouputs.csv*).
- Características de entrada preprocesadas (*Inputs*) de las observaciones del conjunto de Test (*preprocessing_testset_inputs.csv*).
- Etiquetas de salida (*Outputs*) de las observaciones del conjunto de Test (*testset_outputs.csv*).

In [1]:
# pandas library
import pandas as pd
try:
    import pandas as pd
except ImportError as err:
    !pip install pandas
    import pandas as pd


training_inputs = pd.read_csv("./preprocessing_trainset_inputs.csv", index_col="idx")
training_outputs = pd.read_csv("./trainset_ouputs.csv", index_col="idx")
print("Longitud del conjunto de entrenamiento:", len(training_inputs))

testing_inputs = pd.read_csv("./preprocessing_testset_inputs.csv", index_col="idx")
testing_outputs = pd.read_csv("./testset_outputs.csv", index_col="idx")
print("Longitud del conjunto de test:", len(testing_inputs))

Longitud del conjunto de entrenamiento: 13758
Longitud del conjunto de test: 5916


### 2.2. Creación del conjunto de validación

Vamos a dividir el conjunto de test original en dos conjuntos de igual tamaño: el conjunto de validación, que emplearemos para seleccionar los hiperparámetros que den mejores resultados; y el conjunto de test propiamente dicho, que emplearemos para medir la calidad del modelo.

In [2]:
try:
    from sklearn.model_selection import train_test_split
except ImportError as err:
    !pip install sklearn
    from sklearn.model_selection import train_test_split

SEED=1234

# dividimos el conjunto de test original: testing_inputs
test_inputs, val_inputs = train_test_split(testing_inputs, test_size=0.5, train_size=0.5, random_state=SEED, shuffle=True)

# generamos los outputs correspondientes a los dos conjuntos creados
val_outputs = testing_outputs.loc[val_inputs.index]
print("Longitud del conjunto de validación:", len(val_outputs))

test_outputs = testing_outputs.loc[test_inputs.index]
print("Longitud del conjunto de test:", len(test_outputs))

Longitud del conjunto de validación: 2958
Longitud del conjunto de test: 2958


## 3. Creación y evaluación de un modelo básico

### 3.1. Selección de una métrica de evaluación

Para medir la calidad del modelo que vamos a desarrollar, necesitamos seleccionar una métrica que nos dé un valor que cuantifique la diferencia entre los datos conocidos del conjunto de test y los datos predichos por el modelo. En nuestro caso, vamos a emplear la métrica Root Mean Squared Error (en Scikit-learn está disponible mediante la función "root_mean_squared_error").

![alt text](img/RMSE.png "RMSE")

In [6]:
from sklearn.metrics import root_mean_squared_error


### 3.2. Creación de un modelo básico

**Nota**: se puede añadir al propio pipeline de preprocesado

In [3]:
from sklearn.linear_model import LinearRegression

model_lin_reg = LinearRegression()
model_lin_reg.fit(training_inputs, training_outputs)


LinearRegression()

### 3.3. Evaluación de un modelo básico
#### 3.3.1. Evaluación con el dataset de entrenamiento


In [4]:


training_predictions = model_lin_reg.predict(training_inputs)

print("Primeras 5 observaciones del entrenamiento")
for real, pred in zip(training_predictions[:5].round(-2) ,training_outputs.iloc[:5].values):
    print(f"Valor real: {real}, valor predicho: {pred}. Diferencia en absoluto{abs(real-pred)}. Porcentaje desvío: {(abs(real-pred)*100/real).round(2)}%")


# lin_reg_rmse = root_mean_squared_error(traini)

Primeras 5 observaciones del entrenamiento
Valor real: [280200.], valor predicho: [335400.]. Diferencia en absoluto[55200.]. Porcentaje desvío: [19.7]%
Valor real: [287700.], valor predicho: [470000.]. Diferencia en absoluto[182300.]. Porcentaje desvío: [63.36]%
Valor real: [207700.], valor predicho: [170800.]. Diferencia en absoluto[36900.]. Porcentaje desvío: [17.77]%
Valor real: [140400.], valor predicho: [225000.]. Diferencia en absoluto[84600.]. Porcentaje desvío: [60.26]%
Valor real: [198500.], valor predicho: [155300.]. Diferencia en absoluto[43200.]. Porcentaje desvío: [21.76]%


##### Cálculo de las métricas para todo el dataset de entrenamiento

In [5]:
from sklearn.metrics import root_mean_squared_error

rmse_train = root_mean_squared_error(training_outputs, training_predictions)
print(f"RMSE de entrenamiento con modelo de regresión lineal: {rmse_train.round(2)}")

RMSE de entrenamiento con modelo de regresión lineal: 64102.07


#### Prueba de otro modelo

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model_tree = DecisionTreeRegressor(random_state=SEED)
model_tree.fit(training_inputs, training_outputs)


In [ ]:
training_predictions = model_tree.predict(training_inputs)

print("Primeras 5 observaciones del entrenamiento")
for real, pred in zip(training_predictions[:5].round(-2),training_outputs.iloc[:5].values):
    print(f"Valor real: {real}, valor predicho: {pred}. Diferencia en absoluto{abs(real-pred)}. Porcentaje desvío: {(abs(real-pred)*100/real).round(2)}%")

rmse_train  =root_mean_squared_error(training_outputs, training_predictions)
print(f"RMSE de entrenamiento con modelo de árbol de decisión: {rmse_train.round(2)}")



**¿Qué ha pasado? ¿Cómo puede ser que no tenga ningún error?**

**EJERCICIO**
Explica en una celda de texto qué puede estar pasando para que no exista error



In [ ]:
# EJERCICIO
# Explica en esta celda de texto que puede estar pasando para que no exista error





#### Evaluación con un conjunto de datos no visto previamente

In [ ]:
# Evaluación con el dataset de Validación para el modelo de regresión lineal
val_predictions = model_lin_reg.predict(val_inputs)
rmse_val = root_mean_squared_error(val_outputs, val_predictions)
print(f"RMSE de Validación con modelo de regresión lineal: {rmse_val.round(2)}")

# Evaluación con el dataset de Validación para el modelo de árbol de decisión

val_predictions = model_tree.predict(val_inputs)
rmse_val = root_mean_squared_error(val_outputs, val_predictions)
print(f"RMSE de Validación con el modelo árbol de decisión: {rmse_val.round(2)}")

### selección de características


#### Filtros

#### Embebidos

#### Wrappers

### Optimización de hiperparámetros
Los algoritmos de ML tienen una serie de parámetros de configuración que afectan al entrenamiento del modelo y posteriormente al desempeño de este (ej. LR, momento, etc.). Los parámetros por defecto raras veces son los óptimos, y es necesario optimizarlos para cada problema concreto. Este es un proceso muy costoso computacionalmente, y que requiere muchas veces de recursos dedicados. Existen diferentes técnicas para realizar la optimización (ej. GRID, aleatoria, evolutiva, etc.). La más básica es la optimización basada en una búsqueda GRID.

#### Grid Search

La optimización GRID es la prueba exhaustiva de todas las combinaciones posibles de varios parámetros. Para aplicarla, lo primero que necesitamos es establecer los rangos de valores que queremos probar. La prueba exhaustiva es el equivalente a probar todas las combinaciones empleando bucles “for” anidados. Las optimizaciones de hiperparámetros se hacen y se comparan a través de un conjunto de validación (o empleando validación cruzada), pero nunca contra el conjunto de test).


| ![alt text](img/gridSearch.png "Grid Search")| 
|:--:| 
| **Grid Search**: Búsqueda a través de diferentes valores de dos hiperparámetros. Para cada hiperparámetro se consideran 10 valores diferentes (100 combinaciones distintas). Los contornos azules indican las regiones con mejores resultados, mientras que los rojos son regiones con peores resultados. Fuente de la imagen [wikipedia](https://es.m.wikipedia.org/wiki/Archivo:Hyperparameter_Optimization_using_Grid_Search.svg)|

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = [
{'criterion': ["squared_error", "absolute_error"]},
{'max_features': [0.25, 0.5, 0.75, 1.0]}
]

grid_search = GridSearchCV(estimator=model_tree, param_grid=param_grid,
                           cv=[(np.arange(len(training_inputs)), np.arange(len(training_inputs),len(training_inputs)+ len(val_inputs)))])


x=pd.concat([training_inputs, val_inputs])
y=pd.concat([training_outputs, val_outputs])

results=grid_search.fit(x,y)

print(results)
print(grid_search.best_params_)



#### Random Search


| ![alt text](img/randomSearch.png "Random Search")| 
|:--:| 
| **Random Search**: Búsqueda aleatoria entre diferentes combinaciones de valores para dos hiperparámetros. En este ejemplo se evalúan 100 opciones aleatorias diferentes.Los contornos azules indican las regiones con mejores resultados, mientras que los rojos son regiones con peores resultados. Fuente de la imagen [wikipedia](https://commons.wikimedia.org/wiki/File:Hyperparameter_Optimization_using_Random_Search.svg)|


### Validación cruzada